In [1]:
import pygame
import os
import random

pygame 2.6.1 (SDL 2.28.4, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:

# Zombie class
class Zombie(pygame.sprite.Sprite):
    def __init__(self, x, y, gender='male'):
        super().__init__()
        self.gender = gender    
        self.lifecycle = 'Walk' #Walk, Dead
        self.status = 'Active' #Active, Inactive
        
        self.frames = self.load_zombie_frames(gender)
        self.current_frame = 0
        self.image = self.frames[self.lifecycle][self.current_frame]
        self.rect = self.image.get_rect()
        self.rect.topleft = (x, y)
        self.animation_speed = 1  # Adjust for faster/slower walking animation
        self.frame_counter = 0
        
        self.random_speed()
        
        
    def get_lifecycle(self):
        return self.lifecycle
    
    def make_dead(self): 
        self.speed_x = 0
        self.speed_y = 0
        self.lifecycle = 'Dead'
        self.current_frame = 0
        self.distance = 0
    
        
    # Load zombie images
    # Assuming you have a folder named "assets" containing multiple images (frames) named like "zombie1.png", "zombie2.png", etc.
    def load_zombie_frames(self, gender):
        
        lifecycle_list = ['Walk', 'Dead']
        frames_dict = {}
        for lc in lifecycle_list:         
            frames = []
            for i in range(1, 11):  # Adjust the range based on the number of frames
                frame = pygame.image.load(os.path.join('zombie_files', gender, f'{lc} ({i}).png'))
                frames.append(pygame.transform.scale(frame, (64, 128)))  # Resize if necessary
            frames_dict[lc] = frames
        return frames_dict
    
    def random_speed(self): 
        self.speed_x = random.randint(-6, 6)
        self.speed_y = random.randint(-6, 6)
        self.distance = 0 
        self.animation_speed = 0.1 * (1 + abs(self.speed_x) + abs(self.speed_y))

    def update(self):
        
        #speed logic: randomly change speed after walking distance of 100
        self.distance += abs(self.speed_x) + abs(self.speed_y)
        if self.distance >= 1200:
            self.distance = 0
            self.random_speed()
        
        # Animation logic
        self.frame_counter += self.animation_speed
        if self.frame_counter >= len(self.frames[self.lifecycle]):
            self.frame_counter = 0
                        
        self.current_frame = int(self.frame_counter)

        # Movement logic
        self.rect.x += self.speed_x  # Move zombie to the left at a constant speed
        self.rect.y += self.speed_y  # Move zombie to the left at a constant speed
        
#         print(self.rect.x, self.rect.y, self.rect.left, self.rect.right, self.rect.top, self.rect.bottom  )
        
        # Keep zombie on screen
        if self.rect.left <= 0: 
            self.speed_x = abs(self.speed_x)
        if self.rect.right >= SCREEN_WIDTH: 
            self.speed_x = -1 * abs(self.speed_x)
        if self.rect.top <= 0:
            self.speed_y = abs(self.speed_y)
        if self.rect.bottom >= SCREEN_HEIGHT: 
            self.speed_y = -1 * abs(self.speed_y)  
            
            
        if self.speed_x <= 0:
            self.image = pygame.transform.flip(self.frames[self.lifecycle][self.current_frame], True, False)  # Flip horizontally
        elif self.speed_x > 0:
            self.image = self.frames[self.lifecycle][self.current_frame]  # Keep facing right without flipping
        


In [3]:

# Bullet class
class Bullet(pygame.sprite.Sprite):
    def __init__(self, x, y, gender='male'):
        super().__init__()
        self.status = 'Active' #Active, Inactive
        
        self.image = pygame.image.load("bullet.png")  # Load an image for the gun
        self.image = pygame.transform.scale(self.image, (32, 32))
        self.rect = self.image.get_rect()
        self.rect.topleft = (x, y)
        
        self.random_speed()
        
        shoot_sound = pygame.mixer.Sound("gun_firing_sound.wav")
        shoot_sound.set_volume(1)
        shoot_sound.play()
        
        
    def get_status(self):
        return self.status
    
    
    def random_speed(self): 
        self.speed_x = random.randint(-20, 20)
        self.speed_y = random.randint(-20, 20)
        
        while abs(self.speed_x) + abs(self.speed_y) < 10:
            self.speed_x = random.randint(-20, 20)
            self.speed_y = random.randint(-20, 20)

        

    def update(self):
        
        # Movement logic
        self.rect.x += self.speed_x  # Move zombie to the left at a constant speed
        self.rect.y += self.speed_y  # Move zombie to the left at a constant speed
                
        # Keep zombie on screen
        if self.rect.left <= 0: 
            self.speed_x = abs(self.speed_x)
        if self.rect.right >= SCREEN_WIDTH: 
            self.speed_x = -1 * abs(self.speed_x)
        if self.rect.top <= 0:
            self.speed_y = abs(self.speed_y)
        if self.rect.bottom >= SCREEN_HEIGHT: 
            self.speed_y = -1 * abs(self.speed_y)  
            


In [4]:

# Bullet class
class Shotgun(pygame.sprite.Sprite):
    def __init__(self, x):
        super().__init__()
        self.status = 'Active' #Active, Inactive
        
        self.image = pygame.image.load("shotgun.png")  # Load an image for the gun
        self.image = pygame.transform.scale(self.image, (64, 64))
        self.rect = self.image.get_rect()
        
        print("shotgun rect: ", self.rect)

        self.rect.x = x
        self.rect.y = SCREEN_HEIGHT - self.rect.height
        self.speed = 10
        
        print("shotgun rect: ", self.rect)
        print(dir(self))

            
    def move_left(self): 
        self.rect.x -= self.speed
        
        if self.rect.left <= 0:
            self.rect.left = 0 
            
    def move_right(self):
        self.rect.x += self.speed
        if self.rect.right >= SCREEN_WIDTH: 
            self.rect.x = SCREEN_WIDTH - self.rect.width
    
    def fire_bullet(self): 
        bullet = Bullet(self.rect.x + 8, self.rect.y)
        return bullet
    
            


In [5]:
def zombie_bullet_collision(zombie_group, bullet_group): 
    for zombie in zombie_group: 
        for bullet in bullet_group:
            if not zombie or not bullet:
                continue
            if zombie.rect.colliderect(bullet.rect):
                zombie_group.remove(zombie)
                bullet_group.remove(bullet)

def add_zombie(zombie_group):
    if len(zombie_group)<10:
        x_init = random.randint(0, 1) * SCREEN_WIDTH
        y_init = random.randint(0, SCREEN_HEIGHT)
        zombie = Zombie(x_init, y_init)
        zombie_group.add(zombie)
        

In [ ]:

# Initialize Pygame
pygame.init()

# Set screen dimensions
SCREEN_WIDTH, SCREEN_HEIGHT = 800, 600
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Walking Zombie")

# Define Colors
BLACK = (0, 0, 0)

# Game loop
def main():
    clock = pygame.time.Clock()
    zombie_group = pygame.sprite.Group()
    bullet_group = pygame.sprite.Group()
    shotgun_group = pygame.sprite.Group()

    # Create zombie instance and add to group
    zombie_1 = Zombie(SCREEN_WIDTH, SCREEN_HEIGHT - 128)  # Spawn position
    zombie_2 = Zombie(SCREEN_WIDTH, SCREEN_HEIGHT - 256)  # Spawn position
    zombie_3 = Zombie(SCREEN_WIDTH, SCREEN_HEIGHT - 400)
    zombie_4 = Zombie(SCREEN_WIDTH, SCREEN_HEIGHT - 500)
    zombie_5 = Zombie(SCREEN_WIDTH, SCREEN_HEIGHT - 600)
    zombie_6 = Zombie(SCREEN_WIDTH, SCREEN_HEIGHT - 700)

    zombie_group.add(zombie_1)
    zombie_group.add(zombie_2)
    zombie_group.add(zombie_3)
    zombie_group.add(zombie_4)
    zombie_group.add(zombie_5)
    zombie_group.add(zombie_6)
    
    # Create bullet instance
    bullet_1 = Bullet(50, 50)
    bullet_group.add(bullet_1)

    # create shot gun instance 
    shotgun_1 = Shotgun(SCREEN_WIDTH//2)
    shotgun_group.add(shotgun_1)
    
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                
            # Control gun movement
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT:
                    shotgun_1.move_left()
                if event.key == pygame.K_RIGHT:
                    shotgun_1.move_right()
                if event.key == pygame.K_SPACE:
                    bullet = shotgun_1.fire_bullet()
                    bullet_group.add(bullet)
                    
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            shotgun_1.move_left()
        if keys[pygame.K_RIGHT]:
            shotgun_1.move_right()
                
        zombie_bullet_collision(zombie_group, bullet_group)
        add_zombie(zombie_group)
        
        # Update zombie
        zombie_group.update()
        bullet_group.update()

        # Clear the screen
        screen.fill(BLACK)

        # Draw zombie
        zombie_group.draw(screen)
        bullet_group.draw(screen)
        shotgun_group.draw(screen)

        # Refresh the display
        pygame.display.flip()

        # Frame rate
        clock.tick(30)

    pygame.quit()

if __name__ == "__main__":
    main()


shotgun rect:  <rect(0, 0, 64, 64)>
shotgun rect:  <rect(400, 536, 64, 64)>
['_Sprite__g', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'add', 'add_internal', 'alive', 'fire_bullet', 'groups', 'image', 'kill', 'layer', 'move_left', 'move_right', 'rect', 'remove', 'remove_internal', 'speed', 'status', 'update']


In [ ]:
import random
random.randint(0, 1)